<a href="https://colab.research.google.com/github/Gjeffroy/LLM_finetuning/blob/main/Preparing_training_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
! pip install datasets
! pip install pandas
! pip install openai

In [3]:
from datasets import load_dataset

ds = load_dataset("louisbrulenaudet/code-urbanisme")

README.md:   0%|          | 0.00/6.46k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/1.93M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2417 [00:00<?, ? examples/s]

In [5]:
ds['train']

{'ref': "Code de l'urbanisme, art. L101-1",
 'title_main': "Code de l'urbanisme",
 'texte': "Le territoire français est le patrimoine commun de la nation. Les collectivités publiques en sont les gestionnaires et les garantes dans le cadre de leurs compétences. En vue de la réalisation des objectifs définis à l'article L. 101-2 , elles harmonisent leurs prévisions et leurs décisions d'utilisation de l'espace dans le respect réciproque de leur autonomie.",
 'dateDebut': 1451606400000,
 'dateFin': 32472144000000,
 'num': 'L101-1',
 'id': 'LEGIARTI000031210068',
 'cid': 'LEGIARTI000031210068',
 'type': 'AUTONOME',
 'etat': 'VIGUEUR',
 'nota': '',
 'version_article': '1.0',
 'ordre': 1073741823,
 'conditionDiffere': None,
 'infosComplementaires': None,
 'surtitre': None,
 'nature': 'Article',
 'texteHtml': "<p></p><p><br/>\nLe territoire français est le patrimoine commun de la nation.<br/><br/>\nLes collectivités publiques en sont les gestionnaires et les garantes dans le cadre de leurs com

In [8]:
import pandas as pd

df = pd.DataFrame(ds['train'])

In [15]:
template = """Cite moi l'article {article}"""
df['article_id'] = df['ref'].str.split(',').str[-1].str.replace('art.', '')
df['instruction'] = df['article_id'].map(lambda x: template.format(article=x))
df[['article_id', 'instruction', 'texte']]

,article_id,instruction,texte
0,L101-1,Cite moi l'article L101-1,Le territoire français est le patrimoine commu...
1,L101-2,Cite moi l'article L101-2,Dans le respect des objectifs du développement...
2,L101-2-1,Cite moi l'article L101-2-1,L'atteinte des objectifs mentionnés au 6° bis ...
3,L101-3,Cite moi l'article L101-3,La réglementation de l'urbanisme régit l'utili...
4,L102-1,Cite moi l'article L102-1,L'autorité administrative compétente de l'Etat...
...,...,...,...
2412,A614-2,Cite moi l'article A614-2,Les architectes-conseils et les paysagistes-co...
2413,A614-3,Cite moi l'article A614-3,Les dépenses correspondants à la rémunération ...
2414,A614-4,Cite moi l'article A614-4,Les dispositions des articles A. 614-1 à A. 61...
2415,A431-10 Annexe,Cite moi l'article A431-10 Annexe,Attestation du contrôleur technique établissan...


In [34]:
prompt = """
Imagine une question dont la reponse se trouve dans le TEXTE suivant

TEXTE: \\n {}
"""

In [37]:
test = prompt.format(df['texte'][0])
test

"\nImagine une question dont la reponse se trouve dans le texte suivant\n\nTEXTE Le territoire français est le patrimoine commun de la nation. Les collectivités publiques en sont les gestionnaires et les garantes dans le cadre de leurs compétences. En vue de la réalisation des objectifs définis à l'article L. 101-2 , elles harmonisent leurs prévisions et leurs décisions d'utilisation de l'espace dans le respect réciproque de leur autonomie.\n"

In [40]:
# Replace with your actual API key
from google.colab import userdata

import os
from openai import OpenAI

client = OpenAI(
    # This is the default and can be omitted
    api_key=userdata.get('openai_API_key'),
)


def generate_response(prompt):
  chat_completion = client.chat.completions.create(
      messages=[
          {
              "role": "user",
              "content": prompt,
          }
      ],
      model="gpt-3.5-turbo",
  )
  return chat_completion.choices[0].message.content

In [ ]:
# test on the first 5 item of df
for i in range(5):
  print('TEXTE: ' + df['texte'][i])
  print(generate_response(prompt.format(df['texte'][i])))

In [ ]:
df['question_orientée'] = df['texte'].map(lambda x: generate_response(prompt.format(x)))